In [1]:
import pandas as pd
import numpy as np

In [2]:
import time

In [3]:
balance = pd.read_pickle("parsed_data/balance")
cashflow = pd.read_pickle("parsed_data/cashflow")
derived_figures = pd.read_pickle("parsed_data/derived_figures")
income = pd.read_pickle("parsed_data/income")

In [4]:
prices = pd.read_pickle("parsed_data/prices")
derived_shareprices  = pd.read_pickle("parsed_data/derived_shareprices")

In [5]:
Tickers = np.unique(balance.index)

In [6]:
columns = np.array([])
for j in [cashflow, prices, derived_figures, derived_shareprices, income, balance]:
    columns = np.append(columns, j.columns)

In [7]:
unique_columns = np.unique(columns)

In [21]:
database = {}

database["prices"] = prices
database["derived_shareprices"] = derived_shareprices
database["cashflow"] = cashflow
database["derived_figures"] = derived_figures
database["income"] = income
database["balance"] = balance

In [48]:
data = []
for column in unique_columns:
    thing1 = []
    
    percent_valid_mem = 0
    datasheet_mem = 0
    column_mem = 0
    
    try:
       
        for datasheet_name in database.keys():

            if column in database[datasheet_name].columns:

                length  = len(database[datasheet_name][column])

                num_nan = np.sum(np.isnan(database[datasheet_name][column]))
                num_inf = np.sum(np.isinf(database[datasheet_name][column]))
                percent_valid = (1 - ((num_nan + num_inf) / length)) * 100

                if percent_valid >= percent_valid_mem:
                    percent_valid_mem = percent_valid
                    datasheet_mem = datasheet_name
                    column_mem = column
                    
        thing1.append(column_mem)
        thing1.append(percent_valid_mem)
        thing1.append(datasheet_mem)
        
        data.append(thing1)
                    
    except:
        
        thing1.append(column)
        thing1.append(np.nan)
        thing1.append(np.nan)
        data.append(thing1)
        
          

data_df = pd.DataFrame(data)

data_df = data_df.rename(columns = {0: "columns", 1: "percent_valid", 2:"where"})

data_df = data_df.set_index("columns")

data_df.to_csv("dataset_info.csv")

In [6]:
data_on_columns = pd.read_csv("dataset_info.csv", index_col = "columns")

In [ ]:
"""

- EV/EBITDA (enterprise value/ earnings before interest tax depreciation and amortisation) **have**

-   Enterprise Value (Market-Cap + Total Debt - Cash & Cash Equivalents) **have**

        Market Cap **have**

        Total Debt (short term + long term debt ), (found in derived figures, coverage is 99 %, so if want Short Term Debt, can subtract long term debt from total debt)
            Short Term Debt (found in Balance, Coverage is 65%) (88% if we do total debt and subtract long term debt (88%))
            Long Term Debt (found in balance, coverage is 81%)
            Net Debt **dont have** can calculate from total debt - cash and cash equiv
            
    Cash and Cash Equivalents (fond in balance, coverage is 99%) (consists of cash, foreign currency, liquid goods, commercial paper)
    
    Cash, Cash Equivalents & Short Term Investments (99% balance)
    
        Cash from (Repayment of) Debt (found in cashflow, coverage is 84% ), could be useful because if a firm is paying off their debt it is a good sign
        (cash flow from financing activities we dont have)
        
        Cash from (Repurchase of) Equity (found in cashflow, coverage is 85.1%) may be a useful indicator because if a firm is buying back shares it may be a good sign
        
- EBITDA (earnings before interest taxes, depreciation and amortization, coverage is 97% in derived_figures)

- EBIT (make from EBITDA - Depreciation and amortization)  ALSO KNOWN AS OPERATING INCOME (income 100%)
        
- Accounts and Notes Receivable (coverage is 92%, found in balance) (money owed to comapny not payed yet due to purchase made with credit) could be useful as a fraction of EV or market cap. Or adding this to cash and comparing to total debt.
Or dividing it by revenue to find portion of sales that are not paid yet. Then time series could be useful to see if more or less or same portion of sales are being payed for. Would be useful for banks

- Accounts payable (93% coverage and on balance), not too useful unless we calculate the accounts payable turnover ratio = cost of goods sold + Ending inventory - Starting Inventory. We dont have cost of goods sold, (
however we know that cost of goods sold = Total Revenue - Gross profit) but we have costs of Revenue coverage is 90 % and found in balance. Coverage of inventories is 68 % and found in balance. So maybe we could just do
accounts payable / cost of revenue and if this decreases it is an indicator that a company in not paying back their shit

- Book to market value, coverage is 99% found in derived shareprices. (Book value / Market value. Book value is defined as Total Assets - Total Liabilities)

- Capital Expenditure (DONT HAVE BUT CAN BE CALCULATED BY: Change in PP&E + Depreciation)

-Change in fixed assets and intangibles (coverage is 98% found in cashflow) I dont know if this is yoy change or quarterly change

- change in working capital (coverage is 98 % and found in cashflow ), net working capital is current assets - current liabilities

- cost of revenue (coverage is 90% and found in balance) could divide this by MV / BV

- cost of goods sold (dont have but could calculate from revenue - gross profit)

- Current ratio ( = Current assets / Current liabilities) can be very useful (is 99% derived figures)

- Debt ratio = total debt / total assets (coverage is 99% and found in derived figures)

- Depreciation and amortization (not sure what could be used for coverage is 97 %, found in cashflow)

- Dividends paid coverage is 55 % in cashflow, may just turn this into an indicator 1 for paying dividends 0 else

- Dividends per share coverage is same 55%

- EV/FCF coverage is 91% found in derived shareprices

- EV/ sales 98%

- EPS basic and EPS diluted 99%

- Equity before minority interest and equity per share 99% and 98% respectively

- free cash flow 88% FCF/share, FCP/netincome  (FCF is equal to Net cash from operation activities - capital expenditure)  

- Gross profit 98% income. = Total Revenue - cost of goods sold

- Gross profit margin = (revenue - Cost of goods sold) / revenue, coverage is 97% and in derived figures

- Net interest expense (88% income). (could divide this by EBIT (earnings before interest and tax for the interest coverage ratio))

- Liabilities to Equity Ratio (coverage 99% derived figures)

- Long term debt 81 % coverage in balance

- Market cap - 99% coverage

- Misc Long Term Assets and Liabilities coverage both above 95 % found in balance but not sure what could be used for maybe misc / total asset ratio

- Net Cash from Financing Activities (99% cashflow) = cash inflow from issueing debt or equity - dividends paid - cash outflow by repurchase of debt of equity

- Net Cash from Investing Activities (99% cashflow) = cash spent and recieved from non current assets or investments like stocks, property and stuff

- Net Cash from Operating Activities (99% cashflow) = cash inflow and outflow from regular business activites

- Net change in cash (coverage = 100% found in cashflow)

- Net Income (100% income) very useful measure of profitability is also synonymous with earnings so also used in earnngs per share and the like. Net income (Common) is also a metric but not sure what it is because it seems equiv to reg net income,
just fyi Revenue > EBITDA > Net Income

- Net Profit Margin (98% derived_figures) = (Net Income )/ Revenue

- Non-Cash Items (98% cashflow) = (depreciation and amortization mostly) could see when this deviates wildly from depreciation and amortization

- Non-Operating Income (Loss) (99% income ) income or loss not from the basic operating activities

- Operating Expenses (99% income)

- Operating Income (Loss) (100% income) operating income = revenue - COGS - operating expenses - depreciation and amortization (net income removes tax aswell)

- Operating Income/EV (99%, derived_shareprices)

- Operating Margin (97% derived_figures) = Operating Income / Revenue

- Other Equity (85% Balance) not sure what this is, but for "A" it was a negative number and worth 10% of book value (Assets - Liabilities) so odds are it is not insignificant

- Other Financing Activities (85% cashflow) again not sure what this is but probably related to Net Cash From Financing Activities, it can be positive or negative but mostly negative

- Other Long Term Assets / Liabilities (balance, 99%, 97%)

- Other Short Term Assets/ Liabilities (balance, 98% 78% )

- Payables & Accruals 99% balance (this value was actually larger than accounts payable, so if they both represent similar things, maybe this should be used in place of accounts payable)

- Pietroski F-score (95% derived figures ) although some of the metrics used to calc this have lower coverage so idk how that works

- Pretax Income (Loss) (100% income) (useful for seeing how much of an impact tax was having, but probably not much use if im being honest )


-    Price to Book Value  price = book value per share. BVS = book value / shares outstanding   99% coverage and in derived shareprices
     Price to Earnings Ratio (quarterly)
     Price to Earnings Ratio (ttm)  =  price / earnings per share . EPS = net income / shares outstanding             
     Price to Free Cash Flow (quarterly) price / free cashflow per share. FCS = Free cash flow / shares outstanding      (88% derived shareprices)
     Price to Free Cash Flow (ttm)
     Price to Sales Ratio (quarterly) price / (revenue / shares outstanding)    97% coverage
     Price to Sales Ratio (ttm)
     

-	Property, Plant & Equipment, Net (99% balance) this comes up alot and is responsible for large majority of long term assets in most cases. Increase in PP&E may be a good sign

- Research and development (40% income) r and d is an operating expense so could try to estimate this from op expense - COGS and other stuff. Return on r and d = gross prifit of previous period / r and d. May be useful for
specific sectors (pharma) over others

- Retained Earnings (95% balance) earnings retained by the company since the start. Could be useful to see if there is a big change in retained earnings t see what they put the money into: repaying debt, r and d etc

- Return on Assets (100% derived_figures). = Net income / Total Assets. target values vary from sector to sector

- Return on Equity (100% derived figures) = Net income / shareholders equity (which is another name for book value which is equal to Total assets - total liabilities)

- Revenue (98% income) v useful could do revenue / merket cap/bv/ev/ebitda/ebit any other time series revenue

- Sales per share (revenue / outstanding shares) (98% derived figures )

- Selling, General & Administrative (95% income) this represents around 80% of operating expenses, so might be a anomaly when this is really low or really high

- Share Capital & Additional Paid-In Capital (99% balance) Money raised when this goes up or down should indicate more money raised for company so it is significant, when it decreases i dont know what the possible reasons\
for this could be but there are some reasons so it should not be overlooked

- Shares (Basic) and Shares (Diluted) (100% cashflow) basic shares are just the outstanding shares, diluted shares are the outstanding shares if all convertible securities (these weird share bond things and options and some
other stuff) were to be exercised. Might be worth to see where  diluted shares greatly exceed the number of basic shares and that might represent an anomaly

- Shares Outstanding (100% prices) we know what this is

-- Short Term Investment coverage is 31%, but we have cash, cash equiv and short term invest and cash and cash equiv, so just subtract to get short term investment coverage 100%

- Short Term Debt (66% balance) this may be a v useful metric but it has low coverage so we may have to estimate this using other methods (long term debt is 88% and total debt is 100% (idk how that computes) but these are 
the numbers)

- Total Assets (100% balance) v useful (used in book value (shareholders equity) as total assets - total liabilities) also = current assets + noncurrent assets

- Total Current Assets (100% balance). Current assets comprimise of cash and cash equiv  + short term investments + receivables (Accounts and Taxes receivable) + Inventory + other current assets

- Total current liabilities (99% balance) (= accounts and notes payable + short term debt + dividends payable + interest on long term debt + current part of long term debt + income taxes + other stuff)

- Total Debt (100% derived_figures) = long + short term debt but both have less than 100% coverage so not sure how this has the full coverage

- Total Equity (100% balance) = total assets - total liabilities

- Total Liabilities (100% balance) = Current + noncurrent liabilities

- Total Noncurrent Assets (99% balance)

- Total Noncurrent Liabilities (99% balance)

- Working capital  = current assets - current liabilities, we have because I created it in balance

"""

In [ ]:
"""
QC is quarterly change and YC is yearly change

indicators of profit/ profitability ratios:


EBITDA, [QC, YC]
(OPERATING INCOME (Loss) ), EBIT, [QC, YC]

FCF/share, Qc, YC

[FCF/ Revenue, QC, YC] * 
Gross Profit, [QC YC]
Gross Profit Margin, QC, YC  = 1 - (COGS / revenue)   so it is a efficiency metric

[Net Cash from Financing Activities / Cash & Cash Equivalents, QC, YC] *
[Net Cash from Investing Activities/ Cash & Cash Equivalents QC, YC] *
[Net Cash from Operating Activitie/ Cash & Cash Equivalents QC, YC] *

[Net Income, QC, YC]
[Net Income / EV, QC, YC] *
[Net Profit Margin, QC, YC]

[Operating Expenses / Revenue, QC, YC]   instead of Gross Profit Margin *

[Return on Assets, QC, YC] = net income/ total assets
[Return on Equity, QC, YC] = net income / Book Value

[EV / Share Capital & Additional Paid in Capital] *
[Total Assets / Share Capital & Additional Paid in Capital] *




Indicators relating to debt:


Total Debt, QC, YC
Short Term Debt, QC, YC
Current Account to Short term debt ratio
Long Term Debt

***liabilities has much better coverage than debt so might use that instead***
[Cash and Cash Equiv / Total Liabilities, QC, YC]  (substitute for net debt) *
[Retained Earnings / Total Liabilities, QC, YC] *
[Total Current Liabilities / Total Liabilitie, QC, YC] *
[Total Liabilities, QC, YC]
[Total Noncurrent Liabilities / Total Liabilities, QC, YC] (99% balance) *





Indicators of value:

[Enterprise Value, QC, YC]
[EV/EBITDA QC, YC]
[Market Cap]  chnge is just going to be the change in price, unless shares have also changed
[Cash and Cash Equivalents, QC, YC]
[Book to market value QC, YC]
[Working Capital, QC, YC] *
[Current ratio (current assets / current liabilities), QC, YC]
[Total Equity, QC, YC]
[Total assets / Total liabilities, QC, YC] *


[EV/FCF, QC, YC]
[EV/Sales, QC, YC]

[Price to Book Value, QC, YC] price = book value per share. BVS = book value / shares outstanding   99% coverage and in derived shareprices
[Price to Earnings Ratio QC, YC] price / earnings per share . EPS = net income / shares outstanding             
[Price to Free Cash Flow QC, YC] price / free cashflow per share. FCS = Free cash flow / shares outstanding      (88% derived shareprices)
[Price to Sales Ratio QC, YC] price / (revenue / shares outstanding)    97% coverage


[Property, Plant & Equipment, QC, YC]

[Revenue/Market-Cap, QC, YC] *
["Revenue", QC, YC]

[Total Assets, QC, YC]
[Total Current Assets/Total Assets QC, YC] *
[Total Noncurrent Assets/Total Assets] (99% balance) *



Indicators of activity

Money generated by shares?
[change in Share Capital & Additional Paid-In Capital / total share capital, QC, YC]

has there been a buyback or sell recently?
[Cash from (Repurchase of) Equity / Market value, QC, YC]
[change in Shares Outstanding / total shares outstanding, QC, YC]

Have shares been diluted?

[Shares(Diluted) / Shares(Basic), QC, YC]

How much of expense was marketing?
[Selling General and Administrative / (revenue - net income)]

Investing more in marketing?
[Selling General and Administrative (t) / t-1 (QC, YC)]

Increase in SGAA(t, t-1) / Increase in total liabilities(t, t-1)   {{{ceeb}}}s

How much of total expenses is raw equipment?

[COGS (Revenue - Gross Profit) / (revenue - net income)]

Investing more in COGS?

[COGS (t) / COGS (t-1) QC, YC]

How much of total expenses is PP&E?

[Change in PP&E / (revenue - net income)]

Investing more in PP&E?

[change in PP&E / change in PP&E (t-1), Q]

How much of total expenses is Short term investment?

[Short term investment / (revenue - net income)]    ***Cant do this because data unreliable (cash, cash equiv and short term inv - cash and cash equiv is the same as balance["Short term investment"] just the zeros have been converted to nans, probably for
a good reason)

investing more into short term investments?

[Short term investment (t) / short term investment (t-1) YC]
 
 
 
using cash to pay back debt?
[Cash from (Repayment of) Debt / Total Liabilities, QC, YC]

are customers paying on time?
[Accounts and Notes Receivable / revenue, QC, YC]

is company paying on time?
[payables and accruals / cost of revenue, QC, YC] maybe even longer time frame

What is the minority interest like?
[Equity before Minority Interest / Total Equity, YC]

Interest Coverage?
[Operating Income/Interest Expense, Net, QC, YC]

What weird shit a company has?
[Misc Long Term Assets / Total Assets, QC] *
[Misc Long Term Liabilities / Total Liabilities, QC]

What are non cash items like?
[Depriciation and amortization / Non-Cash Items, YC]

Other equity?
[Other Equity / Total Equity, YC]

How much of expenses are being payed in dividends?
Dividends Paid / (Total Revenue - Net Income)




Metrics

Pietroski F-score

Altman Z score (dont have need to calc)
    Altman Z-Score = 1.2A + 1.4B + 3.3C + 0.6D + 1.0E

    Where:

    A = working capital / total assets
    B = retained earnings / total assets
    C = earnings before interest and tax / total assets
    D = market value of equity / total liabilities
    E = sales / total assets
    
Beneish M score: predictor of bankrupcy

(DSRI) DSRI = (Net Receivablest / Salest) / (Net Receivablest-1 / Salest-1)        Change net recievables to accounts and notes receivable
GMI = [(Salest-1 - COGSt-1) / Salest-1] / [(Salest - COGSt) / Salest]          Replace cogs with cost of revenue
AQI = [1 - (Current Assetst + PP&Et + Securitiest) / Total Assetst] / [1 - ((Current Assetst-1 + PP&Et-1 + Securitiest-1) / Total Assetst-1)]   dont even need securities
SGI = Salest / Salest-1
DEPI = (Depreciationt-1/ (PP&Et-1 + Depreciationt-1)) / (Depreciationt / (PP&Et + Depreciationt))   replace Depreciation with depreciation and amortisation but dont know the ratio of depreciation : dep+ amor
SGAI = (SG&A Expenset / Salest) / (SG&A Expenset-1 / Salest-1)
LVGI = [(Current Liabilitiest + Total Long Term Debtt) / Total Assetst] / [(Current Liabilitiest-1 + Total Long Term Debtt-1) / Total Assetst-1]   ratio of long term debt to noncurrent liabilities is 0.6864 and coverage is 99%
TATA = (Net Income - Cashflow operations - Cashflow investments) / Total assets

M-score = −4.84 + 0.92 × DSRI + 0.528 × GMI + 0.404 × AQI + 0.892 × SGI + 0.115 × DEPI −0.172 × SGAI + 4.679 × TATA − 0.327 × LVGI
The threshold value is -1.78 for the model whose coefficients are reported above. (see Beneish 1999, Beneish, Lee, and Nichols 2013, and Beneish and Vorst 2020).

If M-score is less than -1.78, the company is unlikely to be a manipulator. For example, an M-score value of -2.50 suggests a low likelihood of manipulation.
If M-score is greater than −1.78, the company is likely to be a manipulator. For example, an M-score value of -1.50 suggests a high likelihood of manipulation.

Benfords Law for fraud detection


    







Which sector

How is the economy itself performing?
return 40 of sp500
average return_40 of stocks

stdev of sp500 return 10s
stdev of stock return 10s








"""

In [7]:
fundamental_features = pd.DataFrame()
fundamental_features["Ticker"] = balance.index
fundamental_features = fundamental_features.set_index(["Ticker"])
fundamental_features["Publish Date"] = balance["Publish Date"]

In [8]:
fundamental_features_prices = pd.DataFrame()
fundamental_features_prices["Ticker"] = prices.index
fundamental_features_prices = fundamental_features_prices.set_index(["Ticker"])
fundamental_features_prices["Date"] = prices["Date"]

In [9]:
Date_test = np.array([])
start_time = time.perf_counter()
for i in np.arange(int(prices.loc["A"]["Date"].values[0].split("-")[0]), int(prices.loc["A"]["Date"].values[-1].split("-")[0]) + 1):
    for q in ['01', '02', "03", "04", "05", "06", "07", "08", "09", "10", '11', '12']:
        
        for j in ['01', '02', "03", "04", "05", "06", "07", "08", "09", "10", '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']:
            
            Date_test = np.append(Date_test, str(i) + "-" + q + "-" + j  )
            
print(time.perf_counter() - start_time)

0.2810309000000011


In [10]:
Ticker_test = np.array([])
start_time = time.perf_counter()
length = len(Date_test)
for i in Tickers:
    
    Ticker_test = np.append(Ticker_test, [i] * length)
    
print(time.perf_counter() - start_time)

162.92370970000002


In [11]:
Date_test_test = np.array([])
start_time = time.perf_counter()
for i in Tickers:
    Date_test_test = np.append(Date_test_test, Date_test)
        
        
print(time.perf_counter() - start_time)
    

167.6740027


In [12]:
len(Ticker_test), len(Date_test_test)

(8458164, 8458164)

In [13]:
Date_test_df = pd.DataFrame()

In [14]:
Date_test_df["Date"] = Date_test_test

In [15]:
Date_test_df["Ticker"] = Ticker_test

In [16]:
Date_test_df = Date_test_df.set_index(["Ticker", "Date"])

In [17]:
del Date_test_test
del Ticker_test

In [18]:
"""
the names of the feature cols that we already have, and that are not ratios (and therefore want to calculate changes as fractions of original value), and that are not based on price data (in cashflow/balance/income/derived_figures)
"""

feature_cols_already_have = [
    "EBITDA_QC", "EBITDA_YC",
    "Operating Income (Loss)_QC", "Operating Income (Loss)_YC",
    "Gross Profit_QC", "Gross Profit_YC",
    "Net Income", "Net Income_QC", "Net Income_YC", 
    "Total Liabilities", "Total Liabilities_QC", "Total Liabilities_YC",
    "Cash & Cash Equivalents", "Cash & Cash Equivalents_QC", "Cash & Cash Equivalents_YC",
    "Working Capital", "Working Capital_QC", "Working Capital_YC",
    "Total Equity", "Total Equity_QC", "Total Equity_YC",
    "Property, Plant & Equipment, Net", "Property, Plant & Equipment, Net_QC", "Property, Plant & Equipment, Net_YC",
    "Revenue", "Revenue_QC", "Revenue_YC",
    "Total Assets", "Total Assets_QC", "Total Assets_YC",
    "Share Capital & Additional Paid-In Capital", "Share Capital & Additional Paid-In Capital_QC", "Share Capital & Additional Paid-In Capital_YC", 
    "Cost of Revenue", "Cost of Revenue_QC", "Cost of Revenue_YC"
    
]

In [19]:
TF_dict = {}
TF_dict["QC"] = 1
TF_dict["YC"] = 4

In [22]:
for q in feature_cols_already_have:
    
    start_time = time.perf_counter()
    
    split = q.split("_")
    col_name_base = split[0]
    thing = database[data_on_columns.loc[col_name_base]["where"]][col_name_base]
    
    print(q)
    
    if len(split) > 1:
        TF = split[1]
        
        to_append = np.array([])
        for i in Tickers:
            to_append = np.append(to_append, (thing.loc[i] - thing.loc[i].shift(TF_dict[TF]) ) / np.abs(thing.loc[i].shift(TF_dict[TF])) )
            
        fundamental_features[q] = to_append
        
        
        
    else:
        TF = 0
        
        fundamental_features[q] = thing
        
    print(time.perf_counter() - start_time)

EBITDA_QC
1.0445396000000073
EBITDA_YC
1.0109085000000277
Operating Income (Loss)_QC
1.0154387999999699
Operating Income (Loss)_YC
1.0154499000000214
Gross Profit_QC
1.0123528999999962
Gross Profit_YC
1.0219865000000254
Net Income
0.004151999999976397
Net Income_QC
1.0287320999999565
Net Income_YC
1.037755000000061
Total Liabilities
0.0034653999999818552
Total Liabilities_QC
1.0411234999999124
Total Liabilities_YC
1.0331098000000338
Cash & Cash Equivalents
0.0034323000000995307
Cash & Cash Equivalents_QC
1.0107592000000523
Cash & Cash Equivalents_YC
1.0134626000000253
Working Capital
0.003306800000018484
Working Capital_QC
1.0116079000000582
Working Capital_YC
1.017604600000027
Total Equity
0.0032283999998981017
Total Equity_QC
1.0291908000000376
Total Equity_YC
1.0194255000000112
Property, Plant & Equipment, Net
0.0034621999999444597
Property, Plant & Equipment, Net_QC
1.0189603000000034
Property, Plant & Equipment, Net_YC
1.0178940000000694
Revenue
0.004524800000012874
Revenue_QC
1.0

In [23]:
feature_cols_ratios = [
    "Free Cash Flow/Revenue", "Free Cash Flow/Revenue_QC", "Free Cash Flow/Revenue_YC",
    "Net Cash from Financing Activities/Cash & Cash Equivalents", "Net Cash from Financing Activities/Cash & Cash Equivalents_QC", "Net Cash from Financing Activities/Cash & Cash Equivalents_YC",
    "Net Cash from Investing Activities/Cash & Cash Equivalents" ,"Net Cash from Investing Activities/Cash & Cash Equivalents_QC", "Net Cash from Investing Activities/Cash & Cash Equivalents_YC",
    "Net Cash from Operating Activities/Cash & Cash Equivalents", "Net Cash from Operating Activities/Cash & Cash Equivalents_QC", "Net Cash from Operating Activities/Cash & Cash Equivalents_YC",
    "Operating Expenses/Revenue", "Operating Expenses/Revenue_QC", "Operating Expenses/Revenue_YC",
    "Total Assets/Share Capital & Additional Paid-In Capital", "Total Assets/Share Capital & Additional Paid-In Capital_QC", "Total Assets/Share Capital & Additional Paid-In Capital_YC",
    "Cash & Cash Equivalents/Total Liabilities", "Cash & Cash Equivalents/Total Liabilities_QC", "Cash & Cash Equivalents/Total Liabilities_YC",
    "Retained Earnings/Total Liabilities", "Retained Earnings/Total Liabilities_QC", "Retained Earnings/Total Liabilities_YC", 
    "Total Current Liabilities/Total Liabilities", "Total Current Liabilities/Total Liabilities_QC", "Total Current Liabilities/Total Liabilities_YC",
    "Total Noncurrent Liabilities/Total Liabilities", "Total Noncurrent Liabilities/Total Liabilities_QC", "Total Noncurrent Liabilities/Total Liabilities_YC",
    "Total Assets/Total Liabilities", "Total Assets/Total Liabilities_QC", "Total Assets/Total Liabilities_YC",
    "Total Current Assets/Total Assets", "Total Current Assets/Total Assets_QC", "Total Current Assets/Total Assets_YC",
    "Total Noncurrent Assets/Total Assets", "Total Noncurrent Assets/Total Assets_QC","Total Noncurrent Assets/Total Assets_YC",
    "Shares (Diluted)/Shares (Basic)", "Shares (Diluted)/Shares (Basic)_QC", "Shares (Diluted)/Shares (Basic)_YC",
    "Cash from (Repayment of) Debt/Total Liabilities", "Cash from (Repayment of) Debt/Total Liabilities_QC", "Cash from (Repayment of) Debt/Total Liabilities_YC",
    "Accounts & Notes Receivable/Revenue", "Accounts & Notes Receivable/Revenue_QC", "Accounts & Notes Receivable/Revenue_YC",
    "Payables & Accruals/Cost of Revenue", "Payables & Accruals/Cost of Revenue_QC", "Payables & Accruals/Cost of Revenue_YC",
    "Equity Before Minority Interest/Total Equity", "Equity Before Minority Interest/Total Equity_YC",
    "Operating Income (Loss)/Interest Expense, Net", "Operating Income (Loss)/Interest Expense, Net_QC", "Operating Income (Loss)/Interest Expense, Net_YC"
]

In [24]:
for q in feature_cols_ratios:
    
    start_time = time.perf_counter()
    
    base_split = q.split("/")
    numerator = base_split[0]

    split = base_split[1].split("_")
    
    
    denominator = split[0]
    
    thing1 = database[data_on_columns.loc[numerator]["where"]][numerator]
    thing2 = database[data_on_columns.loc[denominator]["where"]][denominator]
    
    print(numerator)
    print(denominator)
    
    
    
    
    if len(split) > 1:
        TF = split[1]
        
        try:
            thing3 = fundamental_features[numerator + "/" + denominator]
        except:
            thing3 = thing1 / thing2
        
        
        
        to_append = np.array([])
        for i in Tickers:
            to_append = np.append(to_append, (thing3.loc[i] - thing3.loc[i].shift(TF_dict[TF]) ) )
            
        fundamental_features[q] = to_append
        
        
        
    else:
        TF = 0
        
        fundamental_features[q] = thing1 / thing2
        
        
    print(time.perf_counter() - start_time)

Free Cash Flow
Revenue
0.022674600000073042
Free Cash Flow
Revenue
0.5381248000001051
Free Cash Flow
Revenue
0.5253490000000056
Net Cash from Financing Activities
Cash & Cash Equivalents
0.008640600000035192
Net Cash from Financing Activities
Cash & Cash Equivalents
0.5216993999999886
Net Cash from Financing Activities
Cash & Cash Equivalents
0.5227084999999079
Net Cash from Investing Activities
Cash & Cash Equivalents
0.008128800000122283
Net Cash from Investing Activities
Cash & Cash Equivalents
0.5217987000000903
Net Cash from Investing Activities
Cash & Cash Equivalents
0.5223814000000857
Net Cash from Operating Activities
Cash & Cash Equivalents
0.009215599999834012
Net Cash from Operating Activities
Cash & Cash Equivalents
0.5232251000002179
Net Cash from Operating Activities
Cash & Cash Equivalents
0.5259552999998505
Operating Expenses
Revenue
0.004627099999879647
Operating Expenses
Revenue
0.5226815999999417
Operating Expenses
Revenue
0.5272921000000679
Total Assets
Share Capit

In [25]:
"""
This is stuff that we already have but:
Net profit margin = net income/ revenue and so i thing it would be better to have the QC and YC be the diff between the current and previous ratios, rather than the fractional change, although not sure what would be best for the model so maybe we cant test this

"""
feature_cols_already_have_anomaly = [
    "Net Profit Margin", "Net Profit Margin_QC", "Net Profit Margin_YC",
    "Return on Assets", "Return on Assets_QC", "Return on Assets_YC",
    "Return on Equity", "Return on Equity_QC", "Return on Equity_YC",
    "Current Ratio", "Current Ratio_QC", "Current Ratio_YC"
    
]

In [26]:
for q in feature_cols_already_have_anomaly:
    
    start_time = time.perf_counter()
    
    split = q.split("_")
    col_name_base = split[0]
    thing = database[data_on_columns.loc[col_name_base]["where"]][col_name_base]
    print(q)
    
    if len(split) > 1:
        TF = split[1]
        
        to_append = np.array([])
        for i in Tickers:
            to_append = np.append(to_append, thing.loc[i] - thing.loc[i].shift(TF_dict[TF]) )
            
        fundamental_features[q] = to_append
        
        
        
    else:
        TF = 0
        
        fundamental_features[q] = thing
        
    print(time.perf_counter() - start_time)

Net Profit Margin
0.004456499999832886
Net Profit Margin_QC
0.5263574000000517
Net Profit Margin_YC
0.5250006999999641
Return on Assets
0.004177300000037576
Return on Assets_QC
0.5248243999999431
Return on Assets_YC
0.5240850000000137
Return on Equity
0.004308700000137833
Return on Equity_QC
0.5490465999998833
Return on Equity_YC
0.5307285999999749
Current Ratio
0.004067600000098537
Current Ratio_QC
0.5265238999998019
Current Ratio_YC
0.5264532000001054


In [27]:
"""
the names of features that are derived from price data
"""
feature_cols_prices = [
    "Enterprise Value", "Enterprise Value_QC", "Enterprise Value_YC",
    "Market-Cap"
]

In [28]:
TF_dict_prices = {}
TF_dict_prices["QC"] = 63
TF_dict_prices["YC"] = 252

In [29]:
for q in feature_cols_prices:
    
    split = q.split("_")
    col_name_base = split[0]
    thing = database[data_on_columns.loc[col_name_base]["where"]][col_name_base]
    start_time = time.perf_counter()
    print(q)
    
    if len(split) > 1:
        TF = split[1]
        
        to_append = np.array([])
        for i in Tickers:
            to_append = np.append(to_append, (thing.loc[i] - thing.loc[i].shift(TF_dict_prices[TF]) ) / np.abs(thing.loc[i].shift(TF_dict_prices[TF])) )
            
        fundamental_features_prices[q] = to_append
        
        
        
    else:
        TF = 0
        
        fundamental_features_prices[q] = thing
        
    print(time.perf_counter() - start_time)

Enterprise Value
0.21529350000014347
Enterprise Value_QC
6.793834199999992
Enterprise Value_YC
6.557371600000124
Market-Cap
0.21179819999997562


In [30]:
"""
feature cols that involve price features but are ratios and that we already have
"""
feature_cols_already_have_anomaly_prices = [
    "Book to Market Value", "Book to Market Value_QC", "Book to Market Value_YC",
    "EV/FCF", "EV/FCF_QC","EV/FCF_YC",
    "EV/Sales", "EV/Sales_QC", "EV/Sales_YC",
    "Price to Book Value", "Price to Book Value_QC", "Price to Book Value_YC",
    "Price to Earnings Ratio (quarterly)", "Price to Earnings Ratio (quarterly)_QC", "Price to Earnings Ratio (quarterly)_YC",
    "Price to Free Cash Flow (quarterly)", "Price to Free Cash Flow (quarterly)_QC", "Price to Free Cash Flow (quarterly)_YC", 
    "Price to Sales Ratio (quarterly)", "Price to Sales Ratio (quarterly)_QC", "Price to Sales Ratio (quarterly)_YC"]

In [31]:
for q in feature_cols_already_have_anomaly_prices:
    
    start_time = time.perf_counter()
    
    split = q.split("_")
    col_name_base = split[0]
    thing = database[data_on_columns.loc[col_name_base]["where"]][col_name_base]
    
    print(col_name_base)
    
    if len(split) > 1:
        TF = split[1]
        
        to_append = np.array([])
        for i in Tickers:
            to_append = np.append(to_append, (thing.loc[i] - thing.loc[i].shift(TF_dict_prices[TF]) ) / np.abs(thing.loc[i].shift(TF_dict_prices[TF])) )
            
        fundamental_features_prices[q] = to_append
        
        
        
    else:
        TF = 0
        
        fundamental_features_prices[q] = thing
        
    print(time.perf_counter() - start_time)

Book to Market Value
0.22007879999978286
Book to Market Value
6.586010900000019
Book to Market Value
6.554823599999963
EV/FCF
0.22217890000001717
EV/FCF
6.578752899999927
EV/FCF
6.58722489999991
EV/Sales
0.2224673000000621
EV/Sales
6.538065599999982
EV/Sales
6.605358900000056
Price to Book Value
0.22146060000000034
Price to Book Value
6.544023500000094
Price to Book Value
6.6126222000000325
Price to Earnings Ratio (quarterly)
0.22193689999994604
Price to Earnings Ratio (quarterly)
6.546371299999919
Price to Earnings Ratio (quarterly)
6.51350350000007
Price to Free Cash Flow (quarterly)
0.21982989999992242
Price to Free Cash Flow (quarterly)
6.550411300000178
Price to Free Cash Flow (quarterly)
6.558839300000045
Price to Sales Ratio (quarterly)
0.22132609999994202
Price to Sales Ratio (quarterly)
6.545307900000125
Price to Sales Ratio (quarterly)
6.582234100000051


In [ ]:
features_exclusive[
        
        [Selling General & Administrative / (revenue - net income)]
    [Change in PP&E / (revenue - net income)]
    
]

In [32]:
to_append = np.array([])
start_time = time.perf_counter()

for i in Tickers:
    to_append = np.append(to_append, income.loc[i,"Selling, General & Administrative"]  / (income.loc[i, "Revenue"] - income.loc[i, "Net Income"]))
    
fundamental_features["Selling, General & Administrative/Total Cost"] = to_append
    
print(time.perf_counter() - start_time)

1.2431048999999348


In [33]:
to_append = np.array([])
start_time = time.perf_counter()

for i in Tickers:
    to_append = np.append(to_append, balance.loc[i,"Property, Plant & Equipment, Net"]  / (income.loc[i, "Revenue"] - income.loc[i, "Net Income"]))
    
fundamental_features["Property, Plant & Equipment, Net/Total Cost"] = to_append
    
print(time.perf_counter() - start_time)

1.243583999999828


In [34]:
"""
The names of the features that are ratios (and therefore want calc not as a percentage) that involve both fundamental and price based data
"""

feature_cols_ratios_prices = [
    "Free Cash Flow/Shares Outstanding", "Free Cash Flow/Shares Outstanding_QC", "Free Cash Flow/Shares Outstanding_YC",
    "Net Income/Enterprise Value", "Net Income/Enterprise Value_QC", "Net Income/Enterprise Value_YC",
    "Enterprise Value/Share Capital & Additional Paid-In Capital", "Enterprise Value/Share Capital & Additional Paid-In Capital_QC", "Enterprise Value/Share Capital & Additional Paid-In Capital_YC",
    "Enterprise Value/EBITDA", "Enterprise Value/EBITDA_QC", "Enterprise Value/EBITDA_YC",
    "Revenue/Market-Cap", "Revenue/Market-Cap_QC", "Revenue/Market-Cap_YC",
    "Cash from (Repurchase of) Equity/Market-Cap", "Cash from (Repurchase of) Equity/Market-Cap_QC", "Cash from (Repurchase of) Equity/Market-Cap_YC"
    
    

]

In [35]:
Date_test_df = Date_test_df.reset_index().set_index(["Ticker", "Date"])

In [36]:
fundamental_features_prices = fundamental_features_prices.reset_index().set_index(["Ticker", "Date"])


In [37]:
prices = prices.reset_index().set_index(["Ticker", "Date"])
derived_shareprices = derived_shareprices.reset_index().set_index(["Ticker", "Date"])
cashflow = cashflow.reset_index().set_index(["Ticker", "Publish Date"])
derived_figures = derived_figures.reset_index().set_index(["Ticker", "Publish Date"])
income = income.reset_index().set_index(["Ticker", "Publish Date"])
balance = balance.reset_index().set_index(["Ticker", "Publish Date"])
    

In [38]:
database = {}

database["prices"] = prices
database["derived_shareprices"] = derived_shareprices
database["cashflow"] = cashflow
database["derived_figures"] = derived_figures
database["income"] = income
database["balance"] = balance

In [39]:
length_of_prices = len(derived_shareprices)
for q in feature_cols_ratios_prices:
    
    start_time = time.perf_counter()
    
    base_split = q.split("/")
    numerator = base_split[0]

    split = base_split[1].split("_")
    
    
    denominator = split[0]
    
    
    
    #thing1 = database[data_on_columns.loc[numerator]["where"]].reset_index().set_index(["Ticker", test_dict_2[data_on_columns.loc[numerator]["where"]]])[numerator]
    #thing2 = database[data_on_columns.loc[denominator]["where"]].reset_index().set_index(["Ticker", test_dict_2[data_on_columns.loc[denominator]["where"]]])[denominator]
    
    thing1 = database[data_on_columns.loc[numerator]["where"]][numerator]
    thing2 = database[data_on_columns.loc[denominator]["where"]][denominator]

    
    num = 1
    
    
    
    
    if len(thing1) < length_of_prices and numerator not in fundamental_features_prices.columns:
        while num in thing1:
            num = num+1
            
        thing1 = thing1.fillna(num)
        
        Date_test_df[numerator] = thing1
        Date_test_df[numerator] = Date_test_df[numerator].fillna(method = "ffill")
        
        fundamental_features_prices[numerator] = Date_test_df[numerator]
        fundamental_features_prices[numerator] = fundamental_features_prices[numerator].replace({num: np.nan})
        
        print(numerator)
        
        
        
    if len(thing2) < length_of_prices and denominator not in fundamental_features_prices.columns:
        while num in thing2:
            num = num+1
            
        thing2 = thing2.fillna(num)
        
        Date_test_df[denominator] = thing2
        Date_test_df[denominator] = Date_test_df[denominator].fillna(method = "ffill")
        
        fundamental_features_prices[denominator] = Date_test_df[denominator]
        fundamental_features_prices[denominator] = fundamental_features_prices[denominator].replace({num: np.nan})
        
        print(denominator)
        

    
    if len(split) > 1:
        TF = split[1]
        
        if len(thing1) < length_of_prices:
            
            thing3 = fundamental_features_prices[numerator] / thing2
            
        elif len(thing2) < length_of_prices:
            thing3 = thing1 / fundamental_features_prices[denominator]
            
        else:
            print(error)
            
        
        to_append = np.array([])
        for i in Tickers:
            to_append = np.append(to_append, (thing3.loc[i] - thing3.loc[i].shift(TF_dict_prices[TF]) ) )
            
        fundamental_features_prices[q] = to_append
        
        
        
    else:
        TF = 0
        
        if len(thing1) < length_of_prices:
            
            fundamental_features_prices[q] = fundamental_features_prices[numerator] / thing2
            
        elif len(thing2) < length_of_prices:
            fundamental_features_prices[q] = thing1 / fundamental_features_prices[denominator]
            
        else:
            print(error)
        
        
        
    print(time.perf_counter() - start_time)

Free Cash Flow
10.833006299999852
42.56183090000013
42.698241199999984
Net Income
6.809211199999936
42.34045930000002
42.39913999999999
Share Capital & Additional Paid-In Capital
7.850009499999942
42.374437899999975
51.86410999999998
EBITDA
8.03943019999997
54.02602779999984
49.79576210000005
Revenue
7.001520199999959
42.39398099999994
42.42973660000007
Cash from (Repurchase of) Equity
7.208902500000022
42.37197779999997
42.24809849999997


In [ ]:
"""
not sure if the "Cash from (Repurchase of) Equity/Market-Cap" cols were done correctly

"""

In [40]:
Date_test_df = Date_test_df.drop(Date_test_df.columns, axis = 1)

In [41]:
fundamental_features_prices = fundamental_features_prices.drop(["Free Cash Flow", "Net Income", "Share Capital & Additional Paid-In Capital", "Cash from (Repurchase of) Equity", "Revenue", "EBITDA"], axis = 1)

In [42]:
fundamental_features_prices.columns

Index(['Enterprise Value', 'Enterprise Value_QC', 'Enterprise Value_YC',
       'Market-Cap', 'Book to Market Value', 'Book to Market Value_QC',
       'Book to Market Value_YC', 'EV/FCF', 'EV/FCF_QC', 'EV/FCF_YC',
       'EV/Sales', 'EV/Sales_QC', 'EV/Sales_YC', 'Price to Book Value',
       'Price to Book Value_QC', 'Price to Book Value_YC',
       'Price to Earnings Ratio (quarterly)',
       'Price to Earnings Ratio (quarterly)_QC',
       'Price to Earnings Ratio (quarterly)_YC',
       'Price to Free Cash Flow (quarterly)',
       'Price to Free Cash Flow (quarterly)_QC',
       'Price to Free Cash Flow (quarterly)_YC',
       'Price to Sales Ratio (quarterly)',
       'Price to Sales Ratio (quarterly)_QC',
       'Price to Sales Ratio (quarterly)_YC',
       'Free Cash Flow/Shares Outstanding',
       'Free Cash Flow/Shares Outstanding_QC',
       'Free Cash Flow/Shares Outstanding_YC', 'Net Income/Enterprise Value',
       'Net Income/Enterprise Value_QC', 'Net Income/Enterpris

In [43]:
Date = prices.loc["A"].index

In [44]:
Date1 = np.array_split(Date, 3)[0]
Date2 = np.array_split(Date, 3)[1]
Date3 = np.array_split(Date, 3)[2]

In [68]:
Date_test_df_Date = Date_test_df.loc["A"].index

In [69]:
Date_test_df_Date1 = Date_test_df_Date[np.where(Date_test_df_Date == Date1[0])[0][0]: np.where(Date_test_df_Date == Date1[-1])[0][0] + 1]
Date_test_df_Date2 = Date_test_df_Date[np.where(Date_test_df_Date == Date2[0])[0][0]: np.where(Date_test_df_Date == Date2[-1])[0][0] + 1]
Date_test_df_Date3 = Date_test_df_Date[np.where(Date_test_df_Date == Date3[0])[0][0]: np.where(Date_test_df_Date == Date3[-1])[0][0] + 1]

In [71]:
Date_test_df2 = Date_test_df.reset_index().set_index(["Date"]).loc[Date_test_df_Date2]
unique_Tickers = np.unique(Date_test_df2["Ticker"])
Date_test_df2 = Date_test_df2.reset_index().set_index(["Ticker"]).loc[unique_Tickers].reset_index().set_index(["Ticker", "Date"])

In [72]:
Date_test_df3 = Date_test_df.reset_index().set_index(["Date"]).loc[Date_test_df_Date3]
unique_Tickers = np.unique(Date_test_df3["Ticker"])
Date_test_df3 = Date_test_df3.reset_index().set_index(["Ticker"]).loc[unique_Tickers].reset_index().set_index(["Ticker", "Date"])

In [73]:
Date_test_df1 = Date_test_df.reset_index().set_index(["Date"]).loc[Date_test_df_Date1]
unique_Tickers = np.unique(Date_test_df1["Ticker"])
Date_test_df1 = Date_test_df1.reset_index().set_index(["Ticker"]).loc[unique_Tickers].reset_index().set_index(["Ticker", "Date"])

In [49]:
fundamental_features.head()

EBITDA_QC  EBITDA_YC  Operating Income (Loss)_QC  \
Ticker Publish Date                                                     
A      2010-10-06          NaN        NaN                         NaN   
       2010-12-20     0.542857        NaN                    0.765217   
       2011-03-09     0.014815        NaN                    0.039409   
       2011-06-07     0.204380        NaN                    0.260664   
       2011-09-07     0.042424   0.965714                    0.056391   

                     Operating Income (Loss)_YC  Gross Profit_QC  \
Ticker Publish Date                                                
A      2010-10-06                           NaN              NaN   
       2010-12-20                           NaN         0.150345   
       2011-03-09                           NaN        -0.021583   
       2011-06-07                           NaN         0.102941   
       2011-09-07                      1.443478        -0.008889   

                     Gross Profit_YC  Net Income  Net Income_QC  \
Ticker Publish Date                                               
A      2010-10-06                NaN   205000000            NaN   
       2010-12-20                NaN   292000000       0.424390   
       2011-03-09                NaN   193000000      -0.339041   
       2011-06-07                NaN   200000000       0.036269   
       2011-09-07           0.230345   330000000       0.650000   

                     Net Income_YC  Total Liabilities  ...  \
Ticker Publish Date                                    ...   
A      2010-10-06              NaN         6290000000  ...   
       2010-12-20              NaN         6460000000  ...   
       2011-03-09              NaN         4705000000  ...   
       2011-06-07              NaN         4688000000  ...   
       2011-09-07         0.609756         4553000000  ...   

                     Return on Assets_QC  Return on Assets_YC  \
Ticker Publish Date                                             
A      2010-10-06                    NaN                  NaN   
       2010-12-20                0.00759                  NaN   
       2011-03-09               -0.00613                  NaN   
       2011-06-07               -0.00087                  NaN   
       2011-09-07                0.01458              0.01517   

                     Return on Equity  Return on Equity_QC  \
Ticker Publish Date                                          
A      2010-10-06             0.07295                  NaN   
       2010-12-20             0.09023              0.01728   
       2011-03-09             0.05780             -0.03243   
       2011-06-07             0.05049             -0.00731   
       2011-09-07             0.07857              0.02808   

                     Return on Equity_YC  Current Ratio  Current Ratio_QC  \
Ticker Publish Date                                                         
A      2010-10-06                    NaN        1.96606               NaN   
       2010-12-20                    NaN        2.00097           0.03491   
       2011-03-09                    NaN        3.27027           1.26930   
       2011-06-07                    NaN        3.20101          -0.06926   
       2011-09-07                0.00562        3.47043           0.26942   

                     Current Ratio_YC  \
Ticker Publish Date                     
A      2010-10-06                 NaN   
       2010-12-20                 NaN   
       2011-03-09                 NaN   
       2011-06-07                 NaN   
       2011-09-07             1.50437   

                     Selling, General & Administrative/Total Cost  \
Ticker Publish Date                                                 
A      2010-10-06                                       -0.386768   
       2010-12-20                                       -0.367601   
       2011-03-09                                       -0.336350   
       2011-06-07                                       -0.3175

In [45]:
fundamental_features = fundamental_features.reset_index().set_index(["Ticker", "Publish Date"])

In [75]:
fundamental_features_prices1 = fundamental_features_prices.reset_index().set_index(["Date"]).loc[Date1]

unique_Tickers = np.unique(fundamental_features_prices1["Ticker"])
print(len(unique_Tickers))
fundamental_features_prices1 = fundamental_features_prices1.reset_index().set_index(["Ticker"]).loc[unique_Tickers].reset_index().set_index(["Ticker", "Date"])

1706


In [76]:
fundamental_features_prices2 = fundamental_features_prices.reset_index().set_index(["Date"]).loc[Date2]

unique_Tickers = np.unique(fundamental_features_prices2["Ticker"])
print(len(unique_Tickers))
fundamental_features_prices2 = fundamental_features_prices2.reset_index().set_index(["Ticker"]).loc[unique_Tickers].reset_index().set_index(["Ticker", "Date"])

1963


In [77]:
fundamental_features_prices3 = fundamental_features_prices.reset_index().set_index(["Date"]).loc[Date3]

unique_Tickers = np.unique(fundamental_features_prices3["Ticker"])
print(len(unique_Tickers))
fundamental_features_prices3 = fundamental_features_prices3.reset_index().set_index(["Ticker"]).loc[unique_Tickers].reset_index().set_index(["Ticker", "Date"])

2001


In [46]:
del prices
del derived_shareprices
del balance
del income
del cashflow
del derived_figures

In [79]:
iterdict = {1: [fundamental_features_prices1, Date_test_df1] , 2: [fundamental_features_prices2, Date_test_df2], 3: [fundamental_features_prices3, Date_test_df3]}

In [80]:
del Date_test_df

In [82]:
fundamental_features.columns[:]

Index(['EBITDA_QC', 'EBITDA_YC', 'Operating Income (Loss)_QC',
       'Operating Income (Loss)_YC', 'Gross Profit_QC', 'Gross Profit_YC',
       'Net Income', 'Net Income_QC', 'Net Income_YC', 'Total Liabilities',
       ...
       'Return on Assets_QC', 'Return on Assets_YC', 'Return on Equity',
       'Return on Equity_QC', 'Return on Equity_YC', 'Current Ratio',
       'Current Ratio_QC', 'Current Ratio_YC',
       'Selling, General & Administrative/Total Cost',
       'Property, Plant & Equipment, Net/Total Cost'],
      dtype='object', length=106)

In [47]:
for j in fundamental_features.columns:
    thing2 = fundamental_features[j]
    start_time = time.perf_counter()
    print(j)
    num = 1
    
    while num in thing2:

        num = num+1

    thing2 = thing2.fillna(num)
    
    Date_test_df[j] = thing2
    Date_test_df[j] = Date_test_df[j].fillna(method = "ffill")
    
    fundamental_features_prices[j] = Date_test_df[j]
    fundamental_features_prices[j] = fundamental_features_prices[j].replace({num: np.nan})
    
    

EBITDA_QC
EBITDA_YC
Operating Income (Loss)_QC
Operating Income (Loss)_YC
Gross Profit_QC
Gross Profit_YC
Net Income
Net Income_QC
Net Income_YC
Total Liabilities
Total Liabilities_QC
Total Liabilities_YC
Cash & Cash Equivalents
Cash & Cash Equivalents_QC
Cash & Cash Equivalents_YC
Working Capital
Working Capital_QC
Working Capital_YC
Total Equity
Total Equity_QC
Total Equity_YC
Property, Plant & Equipment, Net
Property, Plant & Equipment, Net_QC
Property, Plant & Equipment, Net_YC
Revenue
Revenue_QC
Revenue_YC
Total Assets
Total Assets_QC
Total Assets_YC
Share Capital & Additional Paid-In Capital
Share Capital & Additional Paid-In Capital_QC
Share Capital & Additional Paid-In Capital_YC
Cost of Revenue
Cost of Revenue_QC
Cost of Revenue_YC
Free Cash Flow/Revenue
Free Cash Flow/Revenue_QC
Free Cash Flow/Revenue_YC
Net Cash from Financing Activities/Cash & Cash Equivalents
Net Cash from Financing Activities/Cash & Cash Equivalents_QC
Net Cash from Financing Activities/Cash & Cash Equiva

In [48]:
fundamental_features_prices1 = fundamental_features_prices.reset_index().set_index(["Date"]).loc[Date1]
fundamental_features_prices1 = fundamental_features_prices1.reset_index().set_index(["Date", "Ticker"])

In [49]:
fundamental_features_prices1.to_pickle("parsed_data/fundamental_features1")

In [50]:
fundamental_features_prices2 = fundamental_features_prices.reset_index().set_index(["Date"]).loc[Date2]
fundamental_features_prices2 = fundamental_features_prices2.reset_index().set_index(["Date", "Ticker"])

In [51]:
fundamental_features_prices2.to_pickle("parsed_data/fundamental_features2")

In [54]:
del fundamental_features_prices2

In [52]:
del fundamental_features_prices1

In [53]:
fundamental_features_prices3 = fundamental_features_prices.reset_index().set_index(["Date"]).loc[Date3]
fundamental_features_prices3 = fundamental_features_prices3.reset_index().set_index(["Date", "Ticker"])

In [55]:
fundamental_features_prices3.to_pickle("parsed_data/fundamental_features3")

In [62]:
del fundamental_features_prices3

In [83]:
for q in [1, 2, 3]:
    
    for indx, j in enumerate(fundamental_features.columns):        
            
        thing2 = fundamental_features[j]
        
        start_time = time.perf_counter()
        
        print(j)
        
        num = 1
    
        while num in thing2:

            num = num+1
            
        thing2 = thing2.fillna(num)
        
        iterdict[q][1][j] = thing2
        iterdict[q][1][j] = iterdict[q][1][j].fillna(method = "ffill")
        
        iterdict[q][0][j] = iterdict[q][1][j]
        iterdict[q][0][j] = iterdict[q][0][j].replace({num: np.nan})
        
        if not indx%3:
            
            print(time.perf_counter() - start_time)
        
        
    iterdict[q][0].to_pickle("parsed_data/fundamental_features" + str(q))
    
    del iterdict[q]
                
                
        

EBITDA_QC
3.3582353000001604
EBITDA_YC
Operating Income (Loss)_QC
Operating Income (Loss)_YC
2.0975968000002467
Gross Profit_QC
Gross Profit_YC
Net Income
2.09991779999973
Net Income_QC
Net Income_YC
Total Liabilities
2.137912299999698
Total Liabilities_QC
Total Liabilities_YC
Cash & Cash Equivalents
2.1821521000001667
Cash & Cash Equivalents_QC
Cash & Cash Equivalents_YC
Working Capital
2.140244399999574
Working Capital_QC
Working Capital_YC
Total Equity
2.1868243999997503
Total Equity_QC
Total Equity_YC
Property, Plant & Equipment, Net
2.321896800000104
Property, Plant & Equipment, Net_QC
Property, Plant & Equipment, Net_YC
Revenue
2.1925443000000087
Revenue_QC
Revenue_YC
Total Assets
2.158458999999766
Total Assets_QC
Total Assets_YC
Share Capital & Additional Paid-In Capital
2.1679581000003054
Share Capital & Additional Paid-In Capital_QC
Share Capital & Additional Paid-In Capital_YC
Cost of Revenue
2.140936199999942
Cost of Revenue_QC
Cost of Revenue_YC
Free Cash Flow/Revenue
2.190

In [116]:
fundamental_features3 = pd.read_pickle("parsed_data/fundamental_features3")

In [115]:
fundamental_features2 = pd.read_pickle("parsed_data/fundamental_features2")

In [114]:
fundamental_features1 = pd.read_pickle("parsed_data/fundamental_features1")

In [108]:
fundamental_features1 = fundamental_features1.append(fundamental_features2)

In [110]:
del fundamental_features2

In [111]:
fundamental_features1 = fundamental_features1.append(fundamental_features3)

In [112]:
del fundamental_features3

In [ ]:
"""
if the first value is nan, and the 65th value is non nan, then replace all nan vals with the last value of the previous featureset in the same column.
get the date of the first val, so we need all tickers


"""

In [27]:
fundamental_features2.columns[43]

'EBITDA_QC'

In [29]:
prices = pd.read_pickle("parsed_data/prices")

In [32]:
Date = prices.loc["A"]["Date"].values

In [33]:
Date

array(['2011-05-20', '2011-05-23', '2011-05-24', ..., '2021-09-15',
       '2021-09-16', '2021-09-17'], dtype=object)

In [117]:
unqiue_Tickers1 = np.unique(fundamental_features1.reset_index()["Ticker"])

In [118]:
unqiue_Tickers2 = np.unique(fundamental_features2.reset_index()["Ticker"])

In [119]:
unqiue_Tickers3 = np.unique(fundamental_features3.reset_index()["Ticker"])

In [ ]:
for col in fundamental_features3.columns[43:]:
    
    for Ticker in unqiue_Tickers3:
        
        where_na = np.where(fundamental_features3.loc[Ticker, col].iloc[:65].isna())[0]
        
        try:
            if 0 in where_na and len(where_na) < 64:

                first_date_num = np.where(Date == fundamental_features3.loc[Ticker].index[0])[0][0]

                previous_date = Date[first_date_num - 1]

                val_of_previous = fundamental_features2.loc[Ticker, previous_date][col]

                indx_to_change = fundamental_features3.index[where_na]

                fundamental_features3.loc[indx_to_change, col] = val_of_previous
                
        except:
            print(Ticker)
            
            continue

In [121]:
"""
honsetly this shit will take 18 mins x 2 because we have to do for fundamental features3 aswell so makes more sense to just do it all from scratch and learn from this incase it all cant fit in mem.

"""

for col in fundamental_features2.columns[43:]:
    start_time = time.perf_counter()
    
    for Ticker in unqiue_Tickers2:
        
        where_na = np.where(fundamental_features2.loc[Ticker, col].iloc[:65].isna())[0]
        
        try:
            if 0 in where_na and len(where_na) < 64:

                first_date_num = np.where(Date == fundamental_features2.loc[Ticker].index[0])[0][0]

                previous_date = Date[first_date_num - 1]

                val_of_previous = fundamental_features1.loc[Ticker, previous_date][col]

                indx_to_change = fundamental_features2.index[where_na]

                fundamental_features2.loc[indx_to_change, col] = val_of_previous
                
        except:
            
            continue
            
    print(time.perf_counter() - start_time)
            
            
            
            
            
        
        
        
        

KeyboardInterrupt: 